In [1]:
% pylab inline
figsize(10, 8)

import sys
import pickle

sys.path.append("D:/ds/code/implementation")

# Get the detected UK events
with open('D:/ds/pickles/allevents_uk2.pkl', 'rb') as file:  
    docs = pickle.load(file)
    
# Get the true UK events
with open('D:/ds/flooding/floods.pkl', 'rb') as file:
    floods = pickle.load(file)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from datetime import timedelta

# get the min and the maximum date of the events
tmin = min([d['times']['start'] for d in docs]) - timedelta(days=2) 
tmax = max([d['times']['finish'] for d in docs])
tmin, tmax

(datetime.datetime(2015, 12, 6, 16, 0, 0, 423000),
 datetime.datetime(2015, 12, 27, 7, 37, 5, 496000))

In [3]:
with open('ukpoly.pkl', 'rb') as file:
    mp = pickle.load(file)
print("Done")

docs_bd_uk = []
for x in docs:
    do = True
    for p in x['centres']:
        if p is None or not mp.isInside(p[1], p[0]): 
            do = False
            
    if do:
        docs_bd_uk.append(x)
len(docs_bd_uk)

Done


964

In [4]:
steps = []
tcurr = tmin
flood_geo = {}

while tcurr < tmax:
    tnext = tcurr + timedelta(hours=1)
    
    curdocs = []
    for d in docs_bd_uk:
        fint = d['times']['finish']
        stat = d['times']['start']
        
        # In timestep
        if stat < tcurr and fint > tcurr:
            outdoc = {'centres': d['centres'], 'tweets': []}
            for t in d['tweets']:
                if t['time'] < tnext:
                    outdoc['tweets'].append(t)
            curdocs.append(outdoc)
    
    curflds = []
    for f in floods:
        if f['starttime'] < tnext and f['starttime'] > tcurr - timedelta(days=2):
            curflds.append({'alert': f['alert'], 'code': f['code'], 'coordinates': f['coordinates']})
            flood_geo[f['code']] = [f['geometry_data'], f['geometry_type']]
        
    steps.append([tcurr, curdocs, curflds])        
    tcurr = tnext

In [33]:
len(steps)

496

In [5]:
from mpl_toolkits.basemap import Basemap
from Circles.circles import circle
from matplotlib.patches import Polygon as mplPolygon

figsize(16, 8)

for date_id in range(479, 496):# 100
    print("Processing:", date_id)
    figure(date_id)
    bmp = Basemap(projection='merc', lat_0=53.458736, lon_0=-2.2,
        resolution='h', area_thresh = 1000.0,
        urcrnrlat=54.05, urcrnrlon=-1.5,
        llcrnrlat=53.4, llcrnrlon=-3.2, epsg=5520)

    wx = []
    wy = []
    ax = []
    ay = []
    for x in steps[date_id][2]:        
        try:
            x2,y = bmp(x['coordinates'][0], x['coordinates'][1])
        except KeyError:
            continue

        if 'Alert' in x['alert']:
            ax.append(x2)
            ay.append(y)
        else:
            wx.append(x2)
            wy.append(y)

        geodat = eval(flood_geo[x['code']][0])

        if flood_geo[x['code']][1] != 'MultiPolygon':
            geodat = [geodat]

        for o1 in geodat:
            for o2 in o1:
                xy = []
                for i in o2:
                    xy.append(bmp(i[0], i[1]))

                col = 'red' if 'Warning' in x['alert'] else 'orange'
                poly = mplPolygon(xy, facecolor='red' if 'Warning' in x['alert'] else 'orange', alpha=0.8, linewidth=0)
                plt.gca().add_patch(poly)

    bmp.plot(ax, ay, '.', markersize=10, color='orange', label='Alert Center', alpha=0.8)
    bmp.plot(wx, wy, '.', markersize=10, color='red', label='Warning Center', alpha=0.8)

    px = []
    py = []
    for x in steps[date_id][1]:    
        for p in x['centres']:
            if p is None: 
                continue

            x2,y = bmp(p[1], p[0])
            casa = circle(bmp, p[1], p[0], 10)
            pol = mplPolygon([c for c in casa], facecolor='green', alpha=0.2, linewidth=0)
            plt.gca().add_patch(pol)

        for p in x['tweets']:
            x2, y = bmp(p['coordinate'][1], p['coordinate'][0])
            px.append(x2)
            py.append(y)

    bmp.scatter(px, py, marker='x', color='teal', label='Tweet')

    #bmp.drawcoastlines(color="#777777")
    #mp.drawcountries()
    #bmp.fillcontinents(color='#CCCCCC', zorder=0)
    bmp.drawmapboundary()
    #bmp.shadedrelief()
    bmp.arcgisimage(service='World_Shaded_Relief', xpixels = 1500, verbose= True) # Elevation/World_Hillshade 1500
    #bmp.drawrivers(color='#0000ff')

    legend(numpoints=1)

    dformat = "%H:%M %d/%m/%y"
    title(steps[date_id][0].strftime(dformat))
    #title('Merseyside, UK (26/12/2015) - Detected Events and Flood Alerts')
    tight_layout()
    plt.savefig('anim/anim1_' + str(date_id), dpi=100)#, alpha=True)
    plt.close()

Processing: 479
http://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/export?bbox=1087890.2650162876,5936456.2896320885,1205393.050084058,6000235.93555988&bboxSR=5520&imageSR=5520&size=1500,814&dpi=96&format=png32&f=image
Processing: 480
http://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/export?bbox=1087890.2650162876,5936456.2896320885,1205393.050084058,6000235.93555988&bboxSR=5520&imageSR=5520&size=1500,814&dpi=96&format=png32&f=image
Processing: 481
http://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/export?bbox=1087890.2650162876,5936456.2896320885,1205393.050084058,6000235.93555988&bboxSR=5520&imageSR=5520&size=1500,814&dpi=96&format=png32&f=image
Processing: 482
http://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/export?bbox=1087890.2650162876,5936456.2896320885,1205393.050084058,6000235.93555988&bboxSR=5520&imageSR=5520&size=1500,814&dpi=96&format=png32&f=image


In [34]:
from mpl_toolkits.basemap import Basemap
figsize(9, 6)
#f, axarr = plt.subplots(1, segs2, sharey=True)

#for bm_x in range(segs2):
for date_id in range(445, len(steps)):# 100    
    print("Processing:", date_id)
    f, axarr = plt.subplots(1, 2, sharey=True)
    
    bmp = Basemap(projection='merc', lat_0=53.458736, lon_0=-2.2,
        resolution='i', area_thresh = 1000.0,
        urcrnrlat=56, urcrnrlon=2,
        llcrnrlat=49.75, llcrnrlon=-6.75, ax=axarr[0]) 

    wx = []
    wy = []
    ax = []
    ay = []
    for x in steps[date_id][2]:
        try:
            x2,y = bmp(x['coordinates'][0], x['coordinates'][1])
        except KeyError:
            continue

        if 'Alert' in x['alert']:
            ax.append(x2)
            ay.append(y)
        else:
            wx.append(x2)
            wy.append(y)
    
    bmp.plot(wx, wy, '.', markersize=12, color='red', alpha=0.5, label='Warning')
    bmp.plot(ax, ay, '.', markersize=12, color='orange', alpha=0.5, label='Alert')

    bmp.drawcoastlines(color="#777777")
    axarr[0].set_title("Warnings and Alerts")
    
    bmp2 = Basemap(projection='merc', lat_0=53.458736, lon_0=-2.2,
        resolution='i', area_thresh = 1000.0,
        urcrnrlat=56, urcrnrlon=2,
        llcrnrlat=49.75, llcrnrlon=-6.75, ax=axarr[1]) 
    
    px = []
    py = []
    ms = []
    for x in steps[date_id][1]:
        for p in x['centres']:
            if p is None: 
                continue

            if len(x['tweets']) < 25:
                continue

            x2,y = bmp2(p[1], p[0])
            ms.append(25 + (len(x['tweets'])/len(x['centres'])) ** 1/4)
            px.append(x2)
            py.append(y)
            
    bmp2.scatter(px, py, s=ms, marker='.', color='green', alpha=0.5, label='Event')

    bmp2.drawcoastlines(color="#777777")
    axarr[1].set_title("Detected Events")
  
    tight_layout()
    subplots_adjust(wspace=0, hspace=0, bottom=0, top=0.9)

    dformat = "%H:%M %d/%m/%y"
    suptitle(steps[date_id][0].strftime(dformat))

    plt.savefig('anim2/uk-floods_' + str(date_id), dpi=200, alpha=True)
    plt.close()

Processing: 445
Processing: 446
Processing: 447
Processing: 448
Processing: 449
Processing: 450
Processing: 451
Processing: 452
Processing: 453
Processing: 454
Processing: 455
Processing: 456
Processing: 457
Processing: 458
Processing: 459
Processing: 460
Processing: 461
Processing: 462
Processing: 463
Processing: 464
Processing: 465
Processing: 466
Processing: 467
Processing: 468
Processing: 469
Processing: 470
Processing: 471
Processing: 472
Processing: 473
Processing: 474
Processing: 475
Processing: 476
Processing: 477
Processing: 478
Processing: 479
Processing: 480
Processing: 481
Processing: 482
Processing: 483
Processing: 484
Processing: 485
Processing: 486
Processing: 487
Processing: 488
Processing: 489
Processing: 490
Processing: 491
Processing: 492
Processing: 493
Processing: 494
Processing: 495
